<a href="https://colab.research.google.com/github/nguyenhungquang/NLPhw/blob/master/langdetect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.naive_bayes import MultinomialNB
import numpy as np
from nltk.tokenize import sent_tokenize
import pandas as pd
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import codecs
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
def readData(path):
    #dataArray=[]
    with codecs.open(path,'r','utf-8') as f:
        dataArray=f.readlines()
        dataArray=[x.strip() for x in dataArray]
    #print (len(dataArray))
    return dataArray
def tokenize(text):
    sentList=[x for x in sent_tokenize(text)]
    for i,x in enumerate(sentList):
        textLen=len(x.split(" "))
        if i>0 and textLen<3:
            sentList[i-1:i+1]=[sentList[i-1]+sentList[i]]
        """
        if textLen>30:
            sentList[i:i+1]=[x[:15],x[15:]]
        """
    return (sentList,len(sentList))
    #return ([x for x in sent_tokenize(text)],len(sent_tokenize(text)))
def formatData(sentencePath,labelPath,size,tokenized):
    sentArray=readData(sentencePath)
    labelArray=readData(labelPath)
    size=min(size,len(sentArray))
    sentArray=sentArray[:size]
    labelArray=labelArray[:size]
    if tokenized:
        groupLen=0
        for i,x in enumerate(sentArray):
            sentArray[i:i+1],groupLen=tokenize(x)
            labelArray[i:i+1]=[labelArray[i]]*groupLen
        newSize=len(sentArray)
    else:
        newSize=size
    f=np.memmap("data.dat",dtype=object,mode='w+',shape=(2,newSize))
    f[0]=sentArray[:newSize]
    f[1]=labelArray[:newSize]
    f=f.T
    #print (f[0])
    #for i,x in enumerate(f):
    #    f[i:i+1]=tokenize(x[0],x[1])
    #f=[[x,t[1]] for t in f for x in sent_tokenize(t[0])]
    print (len(f))
    dfTable=pd.DataFrame(f)
    del f
    dfTable.columns=['text','label']
    return dfTable

In [3]:
sentPath="x_train.txt"
labelPath="y_train.txt"
dfTable=formatData(sentPath,labelPath,100000,False)
textTrain=dfTable['text']
labelTrain=dfTable['label']
tfidf_vect = TfidfVectorizer(analyzer='char', ngram_range=(1,4))
model = MultinomialNB()
text_clf = Pipeline([('tfidf', tfidf_vect),
                    ('clf', model),
                    ])
text_clf.fit(textTrain,labelTrain)

100000


Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='char', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 4), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [7]:
xTestPath="x_test.txt"
yTestPath="y_test.txt"
testTable=formatData(xTestPath,yTestPath,1000,False)
textTest=testTable['text']
labelTest=testTable['label']
predictions=text_clf.predict(textTest)
accuracy_score(labelTest,predictions)

1000


0.941